In [5]:
import numpy as np
from functools import lru_cache

In [26]:
class TransitionMatrix:
    """Матрица переходов для определения вероятностей событий"""

    def __init__(self, matrix: np.array):
        self.matrix = matrix

    def __str__(self):
        return "Матрица переходов:\n" + f"{self.matrix}"

    def probability_of_transition(self, n: int) -> np.array:
        """Матрица вероятностей переходов из x в y за n шагов"""
        return np.linalg.matrix_power(self.matrix, n)

    def probability_of_first_transition(self, n: int) -> np.array:
        """Матрица вероятностей первого перехода за n шагов"""

        previous_matrix = np.copy(self.matrix)
        for i in range(n - 1):
            previous_matrix = self.matrix_power_skip(self.matrix, previous_matrix)

        return previous_matrix

    def probability_of_last_transition(self, n: int) -> np.array:
        """Матрица вероятносей перехода не более n шагов"""

        previous_matrix = np.copy(self.matrix)
        res = np.copy(self.matrix)

        for i in range(n - 1):
            previous_matrix = self.matrix_power_skip(self.matrix, previous_matrix)
            res += previous_matrix

        return res

    def probability_of_first_return(self, n: int) -> np.array:
        """Матрица вероятностей первого возвращения на n-ом шаге"""
        matrix = np.copy(self.matrix)

        def first(n_: int) -> np.array:
            return self.probability_of_transition_static(matrix, n_) - \
                   sum([first(i) * self.probability_of_transition_static(matrix, n_ - i) for i in range(1, n_)])

        return np.diagonal(first(n))

    def probability_of_last_return(self, n: int) -> np.array:
        matrix = np.copy(self.matrix)
        result = []

        def first(n_):
            res = self.probability_of_transition_static(matrix, n_) - \
                  sum([first(i) * self.probability_of_transition_static(matrix, n_ - i) for i in range(1, n_)])
            result.append(np.diagonal(res))
            return res

        first(n)

        return sum(result)

    def probability_of_state(self, initial_matrix: np.array, n: int) -> np.array:
        """Матрица вероятностей состояний спустя n шагов"""
        return initial_matrix.dot(np.linalg.matrix_power(self.matrix, n))

    def probability_of_steady_states(self) -> np.array:
        """Установившиеся вероятности"""

        matrix = np.copy(self.matrix).transpose()
        np.fill_diagonal(matrix, np.diagonal(matrix) - 1)
        matrix[-1, :] = 1

        vec = np.zeros(len(matrix))
        vec[-1] = 1

        return np.linalg.inv(matrix).dot(vec)

    def average_time(self) -> float:
        """Среднее время возвращения"""
        matrix = np.copy(self.matrix)
        result = []

        @lru_cache(maxsize=None)
        def first(n_):
            res = self.probability_of_transition_static(matrix, n_) - \
                  sum([first(i) * self.probability_of_transition_static(matrix, n_ - i) for i in range(1, n_)])
            result.append(n_ * np.diagonal(res))
            return res

        first(1000)

        return sum(result)

    def average_steps(self) -> np.array:
        """Среднее количество шагов"""

        previous_matrix = np.copy(self.matrix)
        res = np.copy(self.matrix)

        for i in range(1000):
            previous_matrix = self.matrix_power_skip(self.matrix, previous_matrix)
            res += i * previous_matrix

        return res

    @staticmethod
    def matrix_power_skip(left: np.array, right: np.array) -> np.array:

        r = range(len(left))
        return np.array(
            [[sum(left[i, m] * right[m, j] if m != j else 0 for m in r) for j in r] for i in r])

    @staticmethod
    def probability_of_transition_static(matrix: np.array, n: int) -> np.array:
        return np.linalg.matrix_power(matrix, n)

    
class QueueBranch:
    """Система массового обслуживания для рассчета вероятностей событий"""

    def __init__(self, l: int, m: int, u: int, n: int):
        self.l = l
        self.m = m
        self.u = u
        self.n = n

        s = m + n
        matrix = np.zeros((s + 1, s + 1))
        np.fill_diagonal(matrix[:, 1:], l)
        np.fill_diagonal(matrix[1:, :], [*[i * u for i in range(1, m)], *[m * u for j in range(n + 1)]])

        self.matrix = matrix

    def __str__(self):
        return f"lambda={self.l}\n" + f"m={self.m}\n" + f"u={self.u}\n" + f"n={self.n}\n" + f"{self.matrix}"

    def probability_of_steady_states(self):
        """Установившиеся вероятности"""

        matrix = np.copy(self.matrix).transpose()
        s = len(self.matrix)

        np.fill_diagonal(matrix, [(-1) * sum(matrix[:, i]) for i in range(s)])
        matrix[-1, :] = 1

        vector = np.zeros(s)
        vector[-1] = 1

        return np.linalg.inv(matrix).dot(vector)

    def service_intensity(self) -> tuple:
        """Относительная и абсолютная иненсивность обслуживания"""

        relative = 1 - self.probability_of_failure()
        absolute = relative * self.l

        return relative, absolute

    def probability_of_failure(self) -> float:
        """Вероятность отказа в обслуживании"""
        return self.probability_of_steady_states()[-1]

    def probability_of_skip(self) -> float:
        """Вероятность, что поступающая заявка не будет ждать в очереди"""
        return sum(self.probability_of_steady_states()[:self.m])

    def average_length(self) -> float:
        """Средняя длина в очереди"""
        return sum((i * self.probability_of_steady_states()[self.m + i]) for i in range(1, self.n + 1))

    def average_time(self) -> float:
        """Среднее время в очереди"""
        return sum(((i + 1) / (self.m * self.u) *
                    self.probability_of_steady_states()[self.m + i]) for i in range(self.n))

    def average_busy_channels(self) -> float:
        """Среднее число занятых каналов"""
        return (sum((i * self.probability_of_steady_states()[i]) for i in range(1, self.m + 1)) +
                sum((self.m * self.probability_of_steady_states()[i]) for i in range(self.m + 1, self.m + self.n + 1)))

    def average_wait_time(self) -> float:
        """Среднее время простоя системы массового обслуживания"""

        res = 1 / np.sum(self.matrix, axis=1)
        return res[0]

# Задание 1
Система имеет 14 дискретных состояний. Изменение состояний происходит
в дискретные моменты времени с заданной вероятностью. Схема марковского
процесса изображена на рисунке

![](src/state_graph.png)

In [9]:
transition_matrix = np.array([
        [0.05, 0.41, 0.18, 0.36, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0.27, 0.14, 0, 0, 0, 0, 0.27, 0.32, 0, 0, 0, 0, 0, 0],
        [0.22, 0, 0.05, 0.19, 0, 0.36, 0, 0.08, 0.1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0.34, 0.66, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0.42, 0, 0.57, 0, 0.01, 0, 0, 0, 0, 0],
        [0, 0, 0, 0.09, 0.21, 0.22, 0.21, 0, 0, 0.06, 0.21, 0, 0, 0],
        [0, 0, 0, 0, 0.19, 0.22, 0.17, 0.12, 0, 0, 0.3, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0.38, 0.56, 0.06, 0, 0, 0, 0, 0],
        [0, 0, 0.12, 0, 0, 0, 0, 0.23, 0.3, 0, 0.27, 0.08, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0.11, 0, 0.41, 0.42, 0, 0.06, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0.31, 0, 0.14, 0.36, 0.19, 0],
        [0, 0, 0, 0, 0, 0.27, 0.33, 0, 0, 0, 0.14, 0.04, 0, 0.22],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.65, 0, 0.35, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0.11, 0.36, 0, 0, 0.53]
    ])

tm = TransitionMatrix(transition_matrix)
print(tm)

Матрица переходов:
[[0.05 0.41 0.18 0.36 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.27 0.14 0.   0.   0.   0.   0.27 0.32 0.   0.   0.   0.   0.   0.  ]
 [0.22 0.   0.05 0.19 0.   0.36 0.   0.08 0.1  0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.34 0.66 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.42 0.   0.57 0.   0.01 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.09 0.21 0.22 0.21 0.   0.   0.06 0.21 0.   0.   0.  ]
 [0.   0.   0.   0.   0.19 0.22 0.17 0.12 0.   0.   0.3  0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.38 0.56 0.06 0.   0.   0.   0.   0.  ]
 [0.   0.   0.12 0.   0.   0.   0.   0.23 0.3  0.   0.27 0.08 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.11 0.   0.41 0.42 0.   0.06 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.31 0.   0.14 0.36 0.19 0.  ]
 [0.   0.   0.   0.   0.   0.27 0.33 0.   0.   0.   0.14 0.04 0.   0.22]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.65 0.   0.35 0.  ]
 [0.   0.   0.   0.   0.   0.   

Требуюется определить:
1) Вероятность того, что за 9 шагов система перейдет из состояния 5 в состояние 2  
2) Вероятности состояний системы спустя 7 шагов, если в начальный момент вероятность состояний были следующими<br/> <center>A=(0,07; 0,03; 0,14; 0,14; 0,16; 0,03; 0,06; 0,05; 0; 0,09; 0,02; 0,15; 0; 0,06)</center> 
3) Вероятность первого перехода за 10 шагов из состояния 12 в состояние 4  
4) Вероятность перехода из состояния 13 в состояние 2 не позднее чем за 8 шагов  
5) Среднее количество шагов для перехода из состояния 7 в состояние 14  
6) Вероятность первого возвращения в состояние 6 за 8 шагов  
7) Вероятность возвращения в состояние 6 не позднее чем за 6 шагов  
8) Среднее время возвращения в состояние 2  
9) Установившиеся вероятности  

### 1) Вероятность того, что за 9 шагов система перейдет из состояния 5 в состояние 2

In [11]:
n = 9
state1 = 5
state2 = 2
print("1)", end=" ")
print(tm.probability_of_transition(n)[state1 - 1, state2 - 1])

1) 0.0013983963152081997


### 2) Вероятности состояний системы спустя 7 шагов, если в начальный момент вероятности состояний были следующими
A=(0,07; 0,03; 0,14; 0,14; 0,16; 0,03; 0,06; 0,05; 0; 0,09; 0,02; 0,15; 0; 0,06)

In [12]:
n = 7
A = np.array([0.07, 0.03, 0.14, 0.14, 0.16, 0.03, 0.06, 0.05, 0, 0.09, 0.02, 0.15, 0, 0.06])
print("2)", end=" ")
print(tm.probability_of_state(A, n))

2) [0.00324811 0.00163836 0.01207108 0.01723065 0.12047212 0.0863834
 0.18924168 0.10628574 0.09383809 0.01457595 0.18956774 0.07746487
 0.05485667 0.03312554]


### 3) Вероятность первого перехода за 10 шагов из состояния 12 в состояние 4

In [13]:
n = 10
state1 = 12
state2 = 4
print("3)", end=" ")
print(tm.probability_of_first_transition(n)[state1 - 1, state2 - 1])

3) 0.010714604595470115


### 4) Вероятность перехода из состояния 13 в состояние 2 не позднее чем за 8 шагов

In [14]:
n = 8
state1 = 13
state2 = 2
print("4)", end=" ")
print(tm.probability_of_last_transition(n)[state1 - 1, state2 - 1])

4) 0.007593418179203999


### 5) Среднее количество шагов для перехода из состояния 7 в состояние 14

In [17]:
state1 = 7
state2 = 14
print("5)", end=" ")
print(tm.average_steps()[state1 - 1, state2 - 1])

5) 56.702811058819805


### 6) Вероятность первого возвращения в состояние 6 за 8 шагов

In [18]:
n = 8
target_state = 6
print("6)", end=" ")
print(tm.probability_of_first_return(n)[target_state - 1])

6) 0.0360846370875198


### 7) Вероятность возвращения в состояние 6 не позднее чем за 6 шагов

In [19]:
n = 6
target_state = 6
print("7)", end=" ")
print(tm.probability_of_last_transition(n)[target_state - 1])

7) [0.00615735 0.00161389 0.03769317 0.14719761 0.52883931 0.46673046
 0.74646333 0.20474904 0.28412691 0.10122181 0.66386905 0.32833063
 0.16455144 0.06369556]


### 8) Среднее время возвращения в состояние 2

In [20]:
target_state = 2
print("8)", end=" ")
print(tm.average_time()[target_state - 1])

8) 212.18451600824784


### 9) Установившиеся вероятности

In [21]:
print("9)", end=" ")
print(tm.probability_of_steady_states())

9) [0.00352052 0.00167839 0.01314242 0.0172015  0.10836326 0.08431708
 0.17784797 0.10765456 0.0987634  0.01565626 0.19444268 0.08114629
 0.05828229 0.03798337]


# Задание 2  
Задана система массового обслуживания со следующими характеристиками:  
* Интенсивность поступления λ = 25	
* Каналов обслуживания m = 6
* Интенсиность обслуживания μ = 5
* Максимальный размер очереди n = 6

Изначально требований в системе нет  
a) Составьте граф марковского процесса, запишите систему уравнений Колмогорова и найдите установившиеся вероятности состояний  
b) Найдите вероятность отказа в обслуживании  
c) Найдите относительную и абсолютную интенсивность обслуживания  
d) Найдите среднюю длину в очереди  
e) Найдите среднее время в очереди  
f) Найдите среднее число занятых каналов  
g) Найдите вероятность того, что поступающая заявка не будет ждать в очереди  
h) Найти среднее время простоя системы массового обслуживания  
i) Найти среднее время, когда в системе нет очереди

In [27]:
qb = QueueBranch(25, 6, 5, 6)
print(qb)

lambda=25
m=6
u=5
n=6
[[ 0. 25.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 5.  0. 25.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. 10.  0. 25.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. 15.  0. 25.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 20.  0. 25.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. 25.  0. 25.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 30.  0. 25.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. 30.  0. 25.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. 30.  0. 25.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. 30.  0. 25.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. 30.  0. 25.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 30.  0. 25.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 30.  0.]]


### a) Составьте граф марковского процесса, запишите систему уравнений Колмогорова и найдите установившиеся вероятности состояний

![](src/gmp.png)

![](src/cuk.png)

In [28]:
print("a)", end=" ")
print(qb.probability_of_steady_states())

a) [0.00539705 0.02698527 0.06746318 0.11243864 0.1405483  0.1405483
 0.11712358 0.09760299 0.08133582 0.06777985 0.05648321 0.04706934
 0.03922445]


### b) Найдите вероятность отказа в обслуживании

In [30]:
print("b)", end=" ")
print(qb.probability_of_failure())

b) 0.03922445120067937


### c) Найдите относительную и абсолютную интенсивность обслуживания

In [36]:
print("c)", end=" ")
print("Относительная интенсиновсть = %s; абсолютная интенсивность = %s" % qb.service_intensity())

c) Относительная интенсиновсть = 0.9607755487993206; абсолютная интенсивность = 24.019388719983013


### d) Найдите среднюю длину в очереди

In [37]:
print("d)", end=" ")
print(qb.average_length())

d) 1.1602404387795195


### e) Найдите среднее время в очереди

In [38]:
print("e)", end=" ")
print(qb.average_time())

e) 0.04640961755118078


### f) Найдите среднее число занятых каналов

In [39]:
print("f)", end=" ")
print(qb.average_busy_channels())

f) 4.803877743996605


### g) Найдите вероятность того, что поступающая заявка не будет ждать в очереди

In [40]:
print("g)", end=" ")
print(qb.probability_of_skip())

g) 0.4933807538393408


### h) Найти среднее время простоя системы массового обслуживания  

In [41]:
print("h)", end=" ")
print(qb.average_wait_time())

h) 0.04
